In [1]:
from qondense.tapering import tapering
from qondense.cs_vqe import cs_vqe
from qondense.utils.operator_toolkit import exact_gs_energy
import qondense.utils.qonversion_tools as qonvert
from qondense.utils.ordering_heuristics import ordering_heuristics
import json
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from itertools import combinations

In [2]:
with open('data/molecule_geometries/molecule_data.json') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET', 'CO2_STO-3G_SINGLET', 'O2_STO-3G_SINGLET', 'O3_STO-3G_SINGLET', 'HO

In [3]:
# Set molecule parameters
speciesname = 'SiH4_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

delete_input = True
delete_output = True
cisd=1
ccsd=1
fci =1 # wouldn't advise turning this on over 32 qubits!

# Run pyscf.
molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
calculated_molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=cisd,run_ccsd=ccsd,run_fci=fci)

silane


In [4]:
oh = ordering_heuristics(calculated_molecule)

------------------------------------------------
Information concerning the full system:
------------------------------------------------
Number of qubits in full problem: 26
The Hartree-Fock state is |11111111111111111100000000>
HF   energy = -287.91021323
MP2  energy = -287.95998822
CISD energy = -287.98089294
CCSD energy = -287.98331927
FCI energy  = -287.98400189
------------------------------------------------
Tapering information:
------------------------------------------------
We are able to taper 2 qubits from the Hamiltonian
The symmetry sector is [-1, -1]
The tapered Hartree-Fock state is |111111111111111100000000>
------------------------------------------------
CS-VQE information:
------------------------------------------------
Noncontextual GS energy: -287.9587481206679
Symmetry generators:     ['IIZIIIIIIIIIIIIIIIIIIIII', 'ZIZIZIZIZIZIZIZIZIZIZIZI', 'IZIZIZIZIZIZIZIZIZIZIZIZ', 'IIZIZIZIZIZIZIZIZIZIZIZI', 'IIIZIZIZIZIZIZIZIZIZIZIZ', 'IIIIZIIIIIIIIIIIIIIIIIII', 'IIIIIZIZI

In [5]:
test = oh.noncontextual_set[-1]

In [6]:
print(set([pauli.find('Y') for pauli in list(oh.ham_noncontextual._dict.keys())]))
set([pauli.find('Y') for pauli in list(oh.noncontextual_set)])

{2, -1}


{-1, 2}

In [36]:
test in oh.hamiltonian._dict

False

In [26]:
{op:coeff for op,coeff in oh.hamiltonian._dict.items() 
    if op in oh.noncontextual_set}

{'IIIIIIIIIIIIIIIIIIIIIIII': -177.82100519912066,
 'ZIZIZIZIZIZIZIZIZIZIZIZI': -40.45898411537989,
 'IZIZIZIZIZIZIZIZIZIZIZIZ': -40.45898411537989,
 'ZIIIIIIIIIIIIIIIIIIIIIII': 6.621924701010469,
 'IZIIIIIIIIIIIIIIIIIIIIII': 6.621924701010469,
 'IIZIIIIIIIIIIIIIIIIIIIII': 5.7957394548900725,
 'IIIZIIIIIIIIIIIIIIIIIIII': 5.79573945489007,
 'IIIIZIIIIIIIIIIIIIIIIIII': 5.795739454890076,
 'IIIIIZIIIIIIIIIIIIIIIIII': 5.795739454890073,
 'IIIIIIZIIIIIIIIIIIIIIIII': 5.795739454890068,
 'IIIIIIIZIIIIIIIIIIIIIIII': 5.795739454890068,
 'IIIIIIIIZIIIIIIIIIIIIIII': 1.433281799347229,
 'IIIIIIIIIZIIIIIIIIIIIIII': 1.4332817993472295,
 'IIIIIIIIIIZIIIIIIIIIIIII': 1.1732796698597212,
 'IIIIIIIIIIIZIIIIIIIIIIII': 1.1732796698597223,
 'IIIIIIIIIIIIZIIIIIIIIIII': 1.1732796698597212,
 'IIIIIIIIIIIIIZIIIIIIIIII': 1.1732796698597212,
 'IIIIIIIIIIIIIIZIIIIIIIII': 1.173279669859723,
 'IIIIIIIIIIIIIIIZIIIIIIII': 1.1732796698597228,
 'IIIIIIIIIIIIIIIIZIIIIIII': 0.9814657513023901,
 'IIIIIIIIIIIIIIIIIZIIIIII': 

In [33]:
methods = ["b"]#["a", "b", "c", "d"]#, "e", "f"]
data = {"methods":methods}
for m in methods:
    print('Method: ',m,'\n------------------------------------------')
    data[m] = oh.heuristic_errors(heuristic=m, print_info=True)

data['num_tapered'] = oh.n_taper
data['num_qubits']  = oh.n_qubits
data['stabilizers'] = oh.generators
data['hf_energy']   = oh.hf_energy
data['mp_energy']   = oh.mp_energy
data['cisd_energy'] = oh.cisd_energy
data['ccsd_energy'] = oh.ccsd_energy
data['fci_energy']  = oh.fci_energy

with open('data/cs_vqe_results/'+speciesname+'_heuristic_errors.json', 'w') as outfile:
    json.dump(data, outfile)

Method:  b 
------------------------------------------
Number of qubits simulated: 1
CS-VQE error w.r.t. HF energy:   0.0000000057
CS-VQE error w.r.t. MP2 energy:  0.1782106886
CS-VQE error w.r.t. CISD energy: 0.2008395004
CS-VQE error w.r.t. CCSD energy: 0.2193381450

Number of qubits simulated: 2
CS-VQE error w.r.t. HF energy:  -0.0006526869
CS-VQE error w.r.t. MP2 energy:  0.1775579960
CS-VQE error w.r.t. CISD energy: 0.2001868078
CS-VQE error w.r.t. CCSD energy: 0.2186854524

Number of qubits simulated: 3
CS-VQE error w.r.t. HF energy:  -0.0006561159
CS-VQE error w.r.t. MP2 energy:  0.1775545670
CS-VQE error w.r.t. CISD energy: 0.2001833789
CS-VQE error w.r.t. CCSD energy: 0.2186820235

Number of qubits simulated: 4
CS-VQE error w.r.t. HF energy:  -0.0015313700
CS-VQE error w.r.t. MP2 energy:  0.1766793129
CS-VQE error w.r.t. CISD energy: 0.1993081247
CS-VQE error w.r.t. CCSD energy: 0.2178067694

Number of qubits simulated: 5
CS-VQE error w.r.t. HF energy:  -0.0021639181
CS-VQE er

In [9]:
data

{'methods': ['b'],
 'b': {'heuristic': 'unocc_first',
  1: {'energy': -394.31155567,
   'error': 0.042941989205303344,
   'stab_indices': [15, 16, 17, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  2: {'energy': -394.3120147246691,
   'error': 0.04248293453622409,
   'stab_indices': [16, 17, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  3: {'energy': -394.31201471373436,
   'error': 0.042482945470965205,
   'stab_indices': [17, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  4: {'energy': -394.31302102499467,
   'error': 0.041476634210653174,
   'stab_indices': [13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  5: {'energy': -394.32636459382957,
   'error': 0.028133065375754995,
   'stab_indices': [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  6: {'energy': -394.3263662968394,
   'error': 0.028131362365911627,
   'stab_indices': [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]},
  7: {'energy': -394.3445200483531,
   'error': 0.009977610852217822,
   'stab_indices': [10, 9, 8, 7, 6, 5, 

In [7]:
df_type = 'B97M'
print(f'List of {df_type} density functionals in PySCF:\n')
for df in dft.libxc.XC_CODES.keys():
    if df.find(df_type)!=-1:
        print(df)

List of B97M density functionals in PySCF:



NameError: name 'dft' is not defined

In [ ]:
from pyscf import gto, dft, scf, mp, fci, cc

atom_geom = list(zip(atoms, coords))
print(atom_geom)

mol_sys = gto.M(atom = atom_geom, charge=0, spin=0, basis = 'sto-3g', symmetry=True, unit='Angstrom')
mol_sys.build()
#mf_hf = dft.RKS(mol_hf)
#mf_hf.xc = 'lda,vwn' # default
#mf_hf.xc = 'b3lyp'
#mf_hf = mf_hf.newton() # second-order algortihm
print('Performing Hartree-Fock calculation:')
mol_HFK = scf.RHF(mol_sys)
hfk_energy = mol_HFK.kernel()
print(hfk_energy, '\n')

print('\nPerforming Møller–Plesset perturbation:')
mol_MP2 = mp.MP2(mol_HFK)
mol_MP2.kernel()

print('\nPerforming coupled-cluster singles doubles calculation:')
mol_CCSD = cc.CCSD(mol_HFK)
mol_CCSD.kernel()

print('\nPerforming density functional calculation:')
mol_DFT=dft.RKS(mol_sys)
mol_DFT.xc = 'MGGA_XC_B97M_V'
mol_DFT.kernel()

#print('\nPerforming full-configuration interaction calculation:')
#mol_FCI = fci.FCI(mol_HFK)
#mol_FCI.kernel()
#print(f'E(FCI) = {mol_FCI.e_tot}')

print('\nElectronic structure calculcations complete')

In [ ]:
mol_HFK.e_tot, mol_MP2.e_tot, mol_CCSD.e_tot, mol_DFT.e_tot#, mol_FCI.e_tot

In [ ]:
oh.hf_energy, oh.mp_energy, oh.fci_energy

In [ ]:
help(mol_hf)

In [ ]:
HF_X, LDA_X = .6, .08
B88_X = 1. - HF_X - LDA_X
LYP_C = .81
VWN_C = 1. - LYP_C
mf_hf.xc = f'{HF_X:} * HF + {LDA_X:} * LDA + {B88_X:} * B88, {LYP_C:} * LYP + {VWN_C:} * VWN'
mf_hf.kernel()
mf_hf.xc = 'hf'
mf_hf.kernel()

In [ ]:
molecule_data.fci_energy